In [29]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

import numpy as np
import json

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from google.cloud import bigquery
from witwidget.notebook.visualization import WitWidget, WitConfigBuilder

In [30]:
query="""
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks
FROM
  publicdata.samples.natality
WHERE year > 2000
LIMIT 10000
"""
df = bigquery.Client().query(query).to_dataframe()
df.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks
0,7.109908,False,25,1,38.0
1,7.588311,False,19,1,40.0
2,4.812691,True,35,1,33.0
3,7.251004,True,30,2,38.0
4,6.206013,False,21,1,36.0


In [31]:
df.describe()

,weight_pounds,mother_age,plurality,gestation_weeks
count,9994.000000,10000.000000,10000.000000,9873.000000
mean,7.267343,27.299200,1.035300,38.662919
std,1.345904,6.190841,0.197124,2.560298
min,0.595248,13.000000,1.000000,19.000000
25%,6.563162,22.000000,1.000000,38.000000
50%,7.373360,27.000000,1.000000,39.000000
75%,8.124034,32.000000,1.000000,40.000000
max,13.000660,54.000000,4.000000,47.000000


In [32]:
df['is_male'].value_counts()

True     5097
False    4903
Name: is_male, dtype: int64

In [33]:
df = df.dropna()
df = shuffle(df, random_state=2)

In [34]:
labels = df['weight_pounds']
data = df.drop(columns=['weight_pounds'])
data['is_male'] = data['is_male'].astype(int)

In [35]:
data.head()

,is_male,mother_age,plurality,gestation_weeks
3326,1,27,1,39.0
3717,0,29,1,38.0
8767,0,23,1,38.0
3469,0,24,1,36.0
501,0,21,1,40.0


In [36]:
x,y = data,labels
x_train,x_test,y_train,y_test = train_test_split(x,y)

In [37]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(len(x_train.iloc[0]),)),
    Dense(32, activation='relu'),
    Dense(1)]
)

In [38]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(),
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['mae', 'mse'])

In [39]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 64)                320       
_________________________________________________________________
dense_4 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 33        
Total params: 2,433
Trainable params: 2,433
Non-trainable params: 0
_________________________________________________________________


In [40]:
model.fit(x_train, y_train, epochs=10, validation_split=0.1)

Train on 6661 samples, validate on 741 samples
Epoch 1/10
6661/6661 [==============================] - 2s 284us/sample - loss: 1.6446 - mae: 0.9976 - mse: 1.6446 - val_loss: 1.9050 - val_mae: 1.1030 - val_mse: 1.9050
Epoch 2/10
6661/6661 [==============================] - 1s 111us/sample - loss: 1.5494 - mae: 0.9742 - mse: 1.5494 - val_loss: 1.8195 - val_mae: 1.0727 - val_mse: 1.8195
Epoch 3/10
6661/6661 [==============================] - 1s 119us/sample - loss: 1.4818 - mae: 0.9493 - mse: 1.4818 - val_loss: 1.2264 - val_mae: 0.8543 - val_mse: 1.2264
Epoch 4/10
6661/6661 [==============================] - 1s 117us/sample - loss: 1.4452 - mae: 0.9342 - mse: 1.4452 - val_loss: 3.3128 - val_mae: 1.5463 - val_mse: 3.3128
Epoch 5/10
6661/6661 [==============================] - 1s 109us/sample - loss: 1.4125 - mae: 0.9239 - mse: 1.4125 - val_loss: 1.9435 - val_mae: 1.1310 - val_mse: 1.9435
Epoch 6/10
6661/6661 [==============================] - 1s 114us/sample - loss: 1.3856 - mae: 0.9166 - 

In [41]:
num_examples = 10
predictions = model.predict(x_test[:num_examples])

In [42]:
for i in range(num_examples):
    print('Predicted val: ', predictions[i][0])
    print('Actual val: ',y_test.iloc[i])
    print()

Predicted val:  6.9608555
Actual val:  7.12313568522

Predicted val:  3.63474
Actual val:  4.9383546688

Predicted val:  7.212657
Actual val:  8.68841774542

Predicted val:  7.470297
Actual val:  8.71046397162

Predicted val:  7.1798053
Actual val:  7.3744626639

Predicted val:  7.228162
Actual val:  7.87491199864

Predicted val:  7.0022283
Actual val:  6.34049465512

Predicted val:  6.0019865
Actual val:  5.39912079638

Predicted val:  6.9917645
Actual val:  7.68751907594

Predicted val:  7.097318
Actual val:  8.99926953484



In [43]:
wit_data = pd.concat([x_test, y_test], axis=1)

In [44]:
def custom_predict(examples_to_infer):
    preds = model.predict(examples_to_infer)
    return preds

In [45]:
config_builder = (WitConfigBuilder(wit_data[:500].values.tolist(), data.columns.tolist() + ['weight_pounds'])
  .set_custom_predict_fn(custom_predict)
  .set_target_feature('weight_pounds')
  .set_model_type('regression'))
WitWidget(config_builder, height=800)

WitWidget(config={'model_type': 'regression', 'label_vocab': [], 'feature_names': ['is_male', 'mother_age', 'p…